# Programming Assignment 2: Movie recommendation

- 과제 목표: 뉴럴 네트워크 모델을 설계한 후 모델을 학습하여 각 영화들의 embedding 들을 생성하고, 영화 embedding 을 활용하여 각 사용자에게 맞춤형 영화를 추천

# Notice

<br>

- 과제를 수행하면서 각 task 마다 꼭 주어진 1개의 cell만을 사용할 필요는 없으며, 여러 개의 cell을 추가하여 자유롭게 사용해도 괜찮습니다.
- 과제 수행을 위해 필요한 module이 있다면 추가로 import 해도 괜찮습니다.

In [2]:
! pip install tensorflow

# Import Modules

In [3]:
import warnings, random
import numpy as np
import pandas as pd
import random
import torch # For building network
import torch.nn as nn
import torch.nn.functional as F
from itertools import permutations
from itertools import permutations # For making pairs
warnings.filterwarnings('ignore')

In [4]:
import sys
print(sys.executable)

/Users/vidigummy/opt/anaconda3/envs/RS/bin/python


# Data loading

In [5]:
dir = './MovieLens100K/'
df_ratings = pd.read_csv(dir + 'ratings.csv', usecols=['userId', 'movieId', 'rating'])
df_movies = pd.read_csv(dir + 'movies.csv', usecols=['movieId', 'title', 'genres']) # for title-matching

# Preprocessing data

<br>

> ### Problem 1 (3 points)

<br>

1. df_ratings로 부터 각 사용자들이 본 영화를 기록.
2. 사용자 마다 본 영화 목록을 $(movie1, movie2)$, $(movie2, movie1)$ 과 같이 pair로 생성.
    - 즉, 각 사용자 마다 본 영화 목록에 대해 Permutation을 수행
3. 2번 과정이 끝난 후, random을 이용해 각 pair 순서를 무작위로 shuffle.

In [14]:
#### Your Code Here
df_users = df_ratings['userId'].drop_duplicates()
movies_permutation_result = []
cnt = 0
for i in df_users:
    if(cnt == 20):
        break
    is_user_movies = df_ratings['userId'] == i
    user_movies = df_ratings[is_user_movies]['movieId']
    user_movies_list = user_movies.values.tolist()
    user_movies_permutation = list(permutations(user_movies_list, 2))
    movies_permutation_result.extend(user_movies_permutation)
    cnt += 1
random.shuffle(movies_permutation_result)
# print(len(movies_permutation_result))
print(movies_permutation_result[0:13])
print(len(movies_permutation_result))

[(1278, 2036), (1221, 1466), (2671, 30749), (48394, 2005), (3421, 3706), (508, 60756), (31, 170), (2153, 1625), (97938, 5481), (1258, 1374), (1676, 2888), (4896, 778), (1257, 1193)]
1099200


# Build and train neural networks for generating movie embeddings

<br>

> ### Problem 2 (4 points)

<br>

- 각 영화 임베딩을 구하기 위해 뉴럴 네트워크 모델을 활용하여 multi-class classification 을 수행

- 설계할 신경망의 기본 구조는 **Input Layer - Hidden(Embedding) Layer - Output Layer**.
    - 7주차 강의자료 p.7 신경망 구조 이미지 참고

- 현재 Network를 통해 하고자 하는 task는 multi-class classification.
    - 예: $(movie1, movie2)$ 와 같은 입력 데이터와 정답 출력 데이터를 이용해 모델을 학습
        - Input : $movie1$의 one-hot vector
        - Output : $\widehat{movie2}$의 one-hot vector
        - Compute Loss : $\widehat{movie2}$ 와 $movie2$ 간의 Cross-entropy Loss
- 학습이 완료된 이후에 input layer와 hidden(embedding) layer 사이의 weight matrix $W_{in}$를 movie에 대한 embedding vector로 사용이 가능.
> embedding size(# of hidden units)는 100 이하로 두는 것을 권장. <br>
> embedding layer 다음 hidden layer를 더 추가하여 Genre와 같은 추가 정보를 학습에 활용 할 수도 있음 (필수적으로 고려해야할 사항은 아님).

- 설계한 뉴럴 네트워크 모델의 학습이 완료된 후, 학습된 weight matrix $W_{in}$의 행/열벡터를 각 영화에 대한 embedding vector로 간주하여 영화 embedding 들을 구할 수 있음.

In [63]:
class NnNetwork(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.embedded_layer = nn.Linear(input_size, 100)
        self.hidden_layer = nn.Linear(100, 100)
        self.output_layer = nn.Linear(100,input_size)
        self.relu = nn.ReLU()
        self.activation = nn.Sigmoid()
        self.loss_function = nn.CrossEntropyLoss()
    def forward(self,x, target):
        x = self.embedded_layer(x)
        x = self.relu(x)
        # x = self.hidden_layer(x)
        # x = self.relu(x)
        x = self.output_layer(x)
        output = self.activation(x)
        # print("---output : type -> " + str(type(output)) + " || shape -> "+str(output.shape))
        # print(output)
        # print("---target : type -> " + str(type(target)) + " || shape -> "+str(target.shape))
        # print(target)
        loss = self.loss_function(output, target)
        loss.backward()
        return loss

In [64]:
#### Your Code Here
movies_pair_dataframe = pd.DataFrame(data= movies_permutation_result, columns=['movie1','movie2'])

tensor_movie = torch.as_tensor(movies_permutation_result)
tensor_movie1 = tensor_movie[:,0]
tensor_movie2 = tensor_movie[:,1]
# 임베딩 처리
labels_one = torch.as_tensor(movies_pair_dataframe['movie1'].values)
labels_two = torch.as_tensor(movies_pair_dataframe['movie2'].values)


#unique-mapping movie

labels_one_uni = labels_one.unique() #mapping array
labels_two_uni = labels_two.unique() #mapping array
unique_size = labels_one_uni.shape[0]

print(labels_one.shape) #1099200
print(ohv_one_uni.shape) #1859

ohv_one_uni = F.one_hot(labels_one_uni) 
model = NnNetwork(unique_size)
for epoch in range(40):
    #make one-hot vector(input, target Generation)
    for label, targetInput in tensor_movie:
        zero_tensor = torch.zeros(1, unique_size) # B X 1859
        zero_tensor[0][labels_one_uni == label.nonzero(as_tuple=True)[0]] = 1
        target_tensor = torch.zeros(1, unique_size)
        target_tensor[0][labels_two_uni == targetInput.nonzero(as_tuple=True)[0]] = 1

        #모델 돌리기
        loss = model(zero_tensor, target_tensor)
        # loss.backward()
        if epoch % 4 == 0:
            # 4번마다 로그 출력
            print('Epoch {:4d}/{} loss: {:.6f}'.format(
                epoch, 40, loss.item()
            ))
        break



torch.Size([1099200])
torch.Size([1859, 185136])
Epoch    0/40 Cost: -0.000000
Epoch    4/40 Cost: -0.000000
Epoch    8/40 Cost: -0.000000
Epoch   12/40 Cost: -0.000000
Epoch   16/40 Cost: -0.000000
Epoch   20/40 Cost: -0.000000
Epoch   24/40 Cost: -0.000000
Epoch   28/40 Cost: -0.000000
Epoch   32/40 Cost: -0.000000
Epoch   36/40 Cost: -0.000000


In [ ]:

# print(one_labels)
#여기서부터 학습 시키자
# print(movies_pair_dataframe)
# print(ohe_labels)
# layer = nn.Linear(in_features=ohe_labels.shape[0]*ohe_labels.shape[1], out_features=100)
# for movie1 in ohe_labels:
#     model = nn.Linear(in_features=ohe_labels.shape[1],out_features=100)
#     optimizer = torch.optim.SGD(model.parameters(), lr=0.01) 
#     nb_epochs = 10
#     # print("epochs")
#     # print(movie1.shape)
#     # print(ohe_labels.shape[1])
#     # print(type(movie1))
#     for epoch in range(nb_epochs+1):
#         print(torch.FloatTensor(movie1))
#         hidden = model(torch.FloatTensor(movie1))
#         output_model = nn.Linear(in_features=100, out_features=ohe_labels.shape[1])
#         prediction = model(output_model)
#         cross_entropy_loss = nn.CrossEntropyLoss()
#         cost = cross_entropy_loss(output_model, torch.FloatTensor(ohe2_labels))
#         optimizer.zero_grad()
#         # 비용 함수를 미분하여 gradient 계산
#         cost.backward() # backward 연산
#         # W와 b를 업데이트
#         optimizer.step()
#         if epoch % 1 == 0:
#             # 100번마다 로그 출력
#             print('Epoch {:4d}/{} Cost: {:.6f}'.format(
#                 epoch, nb_epochs, cost.item()
#             ))

# Recommend customized movies to user

<br>

> ### Problem 3 (3 points)

<br>

- 임의의 한명의 사용자에 대하여 해당 사용자가 봤던 영화 n개에 대해 **통합된 embedding vector**를 생성.
    - n개의 embedding vector들에 대해, element-wise한 계산을 통해 통합된 하나의 embedding vector를 생성.
    - 이 embedding vector는 해당 사용자의 전반적인 영화 시청 성향을 나타내는 embedding vector로 간주할 수 있음.
    - 즉, **사용자 1명 당 1개의 embedding vector**를 가짐.
- 통합된 embedding vector와 학습된 weight matrix $W_{in}$의 모든 영화 embedding vector들 간의 유사도를 계산.
- 그 중 유사도가 높은 (top n) 영화들을 선정, 사용자에게 추천.
    > Recommended format : MovieId, Title, Genre, Similarity 가 포함된 형식

In [8]:
#### Your Code Here
